In [ ]:
import time
import pandas as pd
import traceback
from requests import get
from bs4 import BeautifulSoup

BASE_URL = "https://godotengine.org"

asset_dict = {}
failed = set()

In [ ]:
for p in range(24):
    page_url = f'{BASE_URL}/asset-library/asset?max_results=100&page={p}&sort=updated'
    print(page_url)
    bs = BeautifulSoup(get(page_url).text)
    for item in bs.select('.asset-item'):
        header = item.select_one('.asset-header')
        asset_url = BASE_URL + header.get('href')
        # print(asset_url)
        if asset_url in asset_dict:
            continue
        try:
            bs_i = BeautifulSoup(get(asset_url).text)
            repo_url = bs_i.select_one('.container a.btn-default').get('href')
            if 'github' in repo_url:
                repo_url = '/'.join(repo_url.split('/', 5)[:5])
                bs_r = BeautifulSoup(get(repo_url).text)
                stars = bs_r.select_one('.js-social-count').text
            elif 'gitlab' in repo_url:
                bs_r = BeautifulSoup(get(repo_url).text)
                stars = bs_r.select_one('.star-count').text
            else:
                starts = "0"
            asset_dict[asset_url] = {
                'name': item.select_one('.asset-title h4').text,
                'asset_url': asset_url,
                'repo_url': repo_url,
                'stars': stars,
                'godot_version': item.select_one('.asset-tags .label-info').text,
                'last_updated': item.select_one('.asset-footer span').text.rsplit('|')[-1],
            }
        except Exception as e:
            failed.add(asset_url)
            traceback.print_exc()
        time.sleep(0.2)

In [ ]:
df = pd.DataFrame.from_records(list(asset_dict.values()))
df["stars"] = pd.to_numeric(df["stars"].apply(lambda x: x.replace(".", "").replace("k", "00")))
df.sort_values("stars", ascending=False).to_csv("index.csv")

In [ ]:
len(asset_dict)